# Grading students' submissions

--- 

## Grading on the containerized grading server (preferred option)

Grading on our containerized grading server, where students' submissions are executed in separate Docker containers is the **recommended option for security reasons**.  
To be able to use the grading server, you will need to **send a request for access** to noto-support@groupes.epfl.ch

Once you have been granted the access, you can then copy the following `otter grade` command line in a noto terminal:

`DOCKER_BUILDKIT=1 DOCKER_HOST=ssh://grader@10.95.33.212 otter grade -v -a dist/autograder/assignment-autograder_*.zip -n remote_test2 --ext ipynb --pdfs --containers 1 moodlesubmissions/*/*`

Parameters:
* `-v`: verbose output
* `-a` + path: path to the grader zip in your local folder
* `-n` + name: an assignment name to use in the Docker image tag (free text ; keep it short)
* `--ext ipynb`: nature of the files to grades, "ipynb" for notebooks
* `--pdfs`: generate pdfs of the students' submissions and copy them in the local folder for further manual grading
* `--containers` + number: number of parallel containers to use (to speed up grading in the case of large number of submissions)
* path: path to the students' submissions in your local folder

Other parameters can be added in a separate JSON configuration file, with the default name `otter_config.json`.  
The configuration file can be added explicitly to the command line with the `-c` + path option.  
More information here: https://otter-grader.readthedocs.io/en/latest/workflow/otter_generate/index.html#grading-configurations
or by running `otter grade --help` in the terminal.

Once the grading is complete, the grades can be found in the `final_grades.csv` file.
A PDF is generated from the submitted notebook during the grading, and ends up in the `submission_pdfs` folder.
**Note that there is a currently bug**: all pdfs have the same name if the submitted notebooks have the same name...

---

## Grading locally

For testing purposes, grading locally on noto is possible using  the `grade_submission` API.  
In this notebook, we demonstrate how to iterate over the individual submissions and call the grader, then generate 2 CSV files for each student (1 with the overall grade and 1 with the detailed grading) as well as a moodle-specific CSV file with the results of the grading.  
The individual moodle-specific CSV files are then agregated to build an overall gradebook that can be imported into moodle.


**Limitations:** 
* this method is not secure, to use only for test purposes!
* this method is sequential and may take a lot of time if there are lots of submission

### Importing grading utility functions

> **PENDING**: look at grades vs. points - computation seems complex with multiple cases depending where the points are specified (at level of question or tests or test)  

In [ ]:
from grading_utils import *

### Retrieving the grader

In [ ]:
# Folder in which the assignment and grader has been generated
distributionfolder = "dist"

# Name of the assignment **file**
assignmentname = "assignment"

# Retrieving the grader zip
graderzip = glob.glob(distributionfolder+"/autograder/"+assignmentname+"-autograder_*.zip")[0]
graderzip

### Retrieving students' submission folders

In [ ]:
# Folder in which to find students' submission folders
allsubmissionsfolder = "moodlesubmissions/"

# Listing all the submissions (folders)
submissionlist = glob.glob(allsubmissionsfolder+"/*/")
submissionlist

### Iterating over submissions, calling the grader and storing the results into CSV files  

We iterate over each submission folder and evaluate the submitted notebook with the grader.  
The result of the grading is stored into individual CSV files which are stored in a `gradebook` folder inside the submission folder:
- 1 file with the overall result (overall points/grade)
- 1 file with the messages resulting from the grader (points per question + test messages)
- the original `gradebook` file provided by moodle is updated with the results of the grading

In [ ]:
%%time

# Folder where to find the moodle grading sheets
moodlegradingsheetfolder = "gradebook"

# List of generated moodle data
moodledata = []

# Iterating over submission folders
for submissionfolder in submissionlist:
    
    # Finding the notebook to grade, using recursive exploration ("**/" pattern + recursive=True) to go across potential subfolders /!\ WATCH OUT for execution time...
    submissionfile = glob.glob(submissionfolder+"**/"+assignmentname+".ipynb", recursive=True)[0] # TODO here do some error management
    #print(submissionfile)
    
    # Grading the individual notebook
    graderresultdf, graderdetailsdf = grade_single_submission(submissionfile, graderzip, submissionfolder+"/"+moodlegradingsheetfolder, includetestcasemessages=True, saveinlocalCSV=True)
    #display(graderresultdf)
    #display(graderdetailsdf)
    
    # Reading and updating the associated moodle CSV file for this notebook
    moodlegradebookfile = glob.glob(submissionfolder+"/"+moodlegradingsheetfolder+"/*_grading.csv")[0] # TODO here do some error management
    moodledf = write_moodleCSV_single_submission(moodlegradebookfile, graderresultdf, graderdetailsdf)
    #display(moodledf)
    
    # Collecting all individual moodle information to build the global moodle file
    moodledata.append(moodledf);

> **/!\ If the assignment needs some manual grading**: the instructor should go over the individual files and edit the CSV files to update the grades.
In that case, the script generating the overall grading sheet should read the updated CSV files instead of working with the list of dataframes...  


### Agregating all the results into 1 CSV file for import in moodle


Finally, we aggregate all `gradebook` files together into 1, which can then be uploaded to moodle.

In [ ]:
# Saving all the moodle data to a general moodle CSV grading sheet
moodledatadf = pd.concat(moodledata)
#display(moodledata)
moodledatadf.to_csv(allsubmissionsfolder+"/"+"overall_grading.csv", index=False, quoting=csv.QUOTE_NONNUMERIC); # The quotes are important for the file to be read by moodle